In [2]:
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime

In [3]:
#move working directory up to acces data with relative paths
os.chdir("..")

In [4]:
pwd

'/Users/tjark/Documents/Python/CairoPopulation.nosync/tfc-git'

### Import all files

In [5]:
# from 04_Activity_Chains
profiles = pd.read_csv('data/interim/activitychains/base_profiles.csv')

trips = pd.read_csv('data/interim/activitychains/trips.csv')

one_trip_chains = pd.read_csv('data/interim/activitychains/one_trip_chains.csv')
two_trip_chains = pd.read_csv('data/interim/activitychains/two_trip_chains.csv')
three_trip_chains = pd.read_csv('data/interim/activitychains/three_trip_chains.csv')
four_trip_chains = pd.read_csv('data/interim/activitychains/four_trip_chains.csv')
five_trip_chains = pd.read_csv('data/interim/activitychains/five_trip_chains.csv')
six_trip_chains = pd.read_csv('data/interim/activitychains/six_trip_chains.csv')

# from 02_generate_population
scaled_pop_loc = pd.read_csv('data/interim/scaled_pop_loc.csv')

In [6]:
# from 02_generate_population
scaled_pop_loc = pd.read_csv('data/interim/scaled_pop_loc.csv')

## Attach profiles based on age and sex

#### Data prep

In [7]:
scaled_pop_loc = scaled_pop_loc[['AGE','SEX','ATTSCH','geometry']]

In [8]:
scaled_pop_loc['SEX'] = scaled_pop_loc['SEX'].replace(1, 'male')
scaled_pop_loc['SEX'] = scaled_pop_loc['SEX'].replace(2, 'female')
scaled_pop_loc.rename(columns={'SEX': 'gender', 'ATTSCH': 'educ', 'AGE': 'age'}, inplace=True)

scaled_pop_loc.head(3)

,age,gender,educ,geometry
0,83,female,0.0,POINT (31.331430700000002 29.845431899999987)
1,32,female,0.0,POINT (31.313874 29.8144589)
2,34,female,0.0,POINT (31.3075189 29.860689800000003)


In [9]:
# Delete 0.14% of population as stay at home profiles
stay_home_pop = 0.14
scaled_pop_loc_active = scaled_pop_loc.sample(n=(round(len(scaled_pop_loc)*(1-stay_home_pop))))

In [10]:
scaled_pop_loc_active['age_det'] = scaled_pop_loc_active['age']
conditions = [
    scaled_pop_loc_active['age'] < 18,
    (scaled_pop_loc_active['age'] >= 18) & (scaled_pop_loc_active['age'] <= 25),
    (scaled_pop_loc_active['age'] >= 26) & (scaled_pop_loc_active['age'] <= 35),
    (scaled_pop_loc_active['age'] >= 36) & (scaled_pop_loc_active['age'] <= 45),
    (scaled_pop_loc_active['age'] >= 46) & (scaled_pop_loc_active['age'] <= 55),
    (scaled_pop_loc_active['age'] >= 56) & (scaled_pop_loc_active['age'] <= 65),
    (scaled_pop_loc_active['age'] >= 66) & (scaled_pop_loc_active['age'] <= 75),
]
choices = ['<18', '18-25', '26-35', '36-45', '46-55', '56-65', '66-75']

scaled_pop_loc_active['age'] = np.select(conditions, choices, default='>75')

In [11]:
scaled_pop_loc_active.head()

,age,gender,educ,geometry,age_det
7947229,<18,male,1.0,POINT (31.317190321649655 30.15125319860771),7
12417960,66-75,female,0.0,POINT (31.2503757 30.102899400000002),66
16038424,26-35,male,0.0,POINT (31.1875347 30.0911132),29
16543005,<18,male,1.0,POINT (30.937156700000003 29.925582199999997),6
12599527,46-55,female,0.0,POINT (31.307901 30.138758899999996),54


In [12]:
conditions = [
    scaled_pop_loc_active['educ'] == 0,
    (scaled_pop_loc_active['age_det'] <12) &  scaled_pop_loc_active['educ'] == 1,
    (scaled_pop_loc_active['age_det'] <18) &  scaled_pop_loc_active['educ'] == 1
]
choices = ['work', 'primary', 'secondary']

scaled_pop_loc_active['main_act'] = np.select(conditions, choices, default='uni')

In [13]:
# Prep profiles

In [14]:
profiles.drop(columns=['Unnamed: 0'], inplace=True)

In [15]:
data = {
    'profile_id': ['1605', '1606', '1607', '1608', '1609', '1610','1611','1612'],
    'age': ['<18', '<18','<18', '<18', '>75', '>75', '>75', '>75'],
    'gender': ['male', 'male', 'female', 'female', 'female', 'female', 'male', 'male'],
    'vehicle_owned': [0, 0, 0, 0, 0, 1, 0, 1],
    'trips_count': [2, 3, 2, 3, 2, 2, 2, 2],
    'activities': [1, 2, 1, 2, 1, 1, 1, 1]
}

added_profiles = pd.DataFrame(data)

In [16]:
profiles = profiles.append(added_profiles, ignore_index=True)

/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_9192/2171748251.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  profiles = profiles.append(added_profiles, ignore_index=True)


In [17]:
profiles.head(3)

,profile_id,age,gender,vehicle_owned,trips_count,activities
0,0,18-25,female,0,0,0
1,1,56-65,male,0,2,1
2,2,18-25,female,0,3,2


### Merge profiles with pop

In [18]:
df = scaled_pop_loc_active

In [19]:
df = df.reset_index()

In [20]:
# Define the chunk size
chunksize = 50000

# Create an empty list to store the merged chunks
merged_chunks = []

# Get the total number of chunks
total_chunks = -(-len(df) // chunksize)  # Round up division

# Iterate over the data in chunks
for i in range(total_chunks):
    start = i * chunksize
    end = (i + 1) * chunksize
    # Get the chunk of data
    chunk_df = df[start:end]
    # Perform the merge operation on each chunk
    merged_chunk = pd.merge(chunk_df, profiles, on=['age', 'gender'], how='left')    
    # Append the merged chunk to the list
    merged_chunks.append(merged_chunk)
    print(f'Chunk {i+1} of {total_chunks}')

Chunk 1 of 356
Chunk 2 of 356
Chunk 3 of 356
Chunk 4 of 356
Chunk 5 of 356
Chunk 6 of 356
Chunk 7 of 356
Chunk 8 of 356
Chunk 9 of 356
Chunk 10 of 356
Chunk 11 of 356
Chunk 12 of 356
Chunk 13 of 356
Chunk 14 of 356
Chunk 15 of 356
Chunk 16 of 356
Chunk 17 of 356
Chunk 18 of 356
Chunk 19 of 356
Chunk 20 of 356
Chunk 21 of 356
Chunk 22 of 356
Chunk 23 of 356
Chunk 24 of 356
Chunk 25 of 356
Chunk 26 of 356
Chunk 27 of 356
Chunk 28 of 356
Chunk 29 of 356
Chunk 30 of 356
Chunk 31 of 356
Chunk 32 of 356
Chunk 33 of 356
Chunk 34 of 356
Chunk 35 of 356
Chunk 36 of 356
Chunk 37 of 356
Chunk 38 of 356
Chunk 39 of 356
Chunk 40 of 356
Chunk 41 of 356
Chunk 42 of 356
Chunk 43 of 356
Chunk 44 of 356
Chunk 45 of 356
Chunk 46 of 356
Chunk 47 of 356
Chunk 48 of 356
Chunk 49 of 356
Chunk 50 of 356
Chunk 51 of 356
Chunk 52 of 356
Chunk 53 of 356
Chunk 54 of 356
Chunk 55 of 356
Chunk 56 of 356
Chunk 57 of 356
Chunk 58 of 356
Chunk 59 of 356
Chunk 60 of 356
Chunk 61 of 356
Chunk 62 of 356
Chunk 63 of 356
C

In [22]:
df_merged_temps = []

for i in (range(len(merged_chunks))):
    df_merged_temp = merged_chunks[i].groupby('index').sample(n=1).reset_index(drop=True)
    df_merged_temps.append(df_merged_temp)
    print(f'Grouped and sampled for chunk {i+1} of {total_chunks}.')

Grouped and sampled for chunk 1 of 356.
Grouped and sampled for chunk 2 of 356.
Grouped and sampled for chunk 3 of 356.
Grouped and sampled for chunk 4 of 356.
Grouped and sampled for chunk 5 of 356.
Grouped and sampled for chunk 6 of 356.
Grouped and sampled for chunk 7 of 356.
Grouped and sampled for chunk 8 of 356.
Grouped and sampled for chunk 9 of 356.
Grouped and sampled for chunk 10 of 356.
Grouped and sampled for chunk 11 of 356.
Grouped and sampled for chunk 12 of 356.
Grouped and sampled for chunk 13 of 356.
Grouped and sampled for chunk 14 of 356.
Grouped and sampled for chunk 15 of 356.
Grouped and sampled for chunk 16 of 356.
Grouped and sampled for chunk 17 of 356.
Grouped and sampled for chunk 18 of 356.
Grouped and sampled for chunk 19 of 356.
Grouped and sampled for chunk 20 of 356.
Grouped and sampled for chunk 21 of 356.
Grouped and sampled for chunk 22 of 356.
Grouped and sampled for chunk 23 of 356.
Grouped and sampled for chunk 24 of 356.
Grouped and sampled for c

Grouped and sampled for chunk 199 of 356.
Grouped and sampled for chunk 200 of 356.
Grouped and sampled for chunk 201 of 356.
Grouped and sampled for chunk 202 of 356.
Grouped and sampled for chunk 203 of 356.
Grouped and sampled for chunk 204 of 356.
Grouped and sampled for chunk 205 of 356.
Grouped and sampled for chunk 206 of 356.
Grouped and sampled for chunk 207 of 356.
Grouped and sampled for chunk 208 of 356.
Grouped and sampled for chunk 209 of 356.
Grouped and sampled for chunk 210 of 356.
Grouped and sampled for chunk 211 of 356.
Grouped and sampled for chunk 212 of 356.
Grouped and sampled for chunk 213 of 356.
Grouped and sampled for chunk 214 of 356.
Grouped and sampled for chunk 215 of 356.
Grouped and sampled for chunk 216 of 356.
Grouped and sampled for chunk 217 of 356.
Grouped and sampled for chunk 218 of 356.
Grouped and sampled for chunk 219 of 356.
Grouped and sampled for chunk 220 of 356.
Grouped and sampled for chunk 221 of 356.
Grouped and sampled for chunk 222 

In [26]:
# Concatenate the merged chunks into a single DataFrame
df_merged = pd.concat(df_merged_temps)

In [92]:
print(f'Number of people: {len(df_merged)}.')
print('________________')
print(f"Number of activities: {df_merged['activities'].sum()}.")

Number of people: 17760502.
________________
Number of activities: 22946724.


In [97]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Code to execute
scaled_df = df_merged.iloc[np.repeat(np.arange(len(df_merged)), df_merged['activities'])].reset_index(drop=True)
scaled_df['repetition'] = scaled_df.groupby('index').cumcount() + 1
# Get the current time after the code execution
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
# Print the start and end time
print("End Time:", end_time)
print('–––––––––––––––––––')
print(len(scaled_df))
print('–––––––––––––––––––')
print(scaled_df.head(3))

Start Time: 2023-06-18 11:16:33
End Time: 2023-06-18 11:16:50
–––––––––––––––––––
22946724
–––––––––––––––––––
   index    age gender  educ                                       geometry  \
0    628  36-45   male   0.0  POINT (31.302296500000004 29.836736299999988)   
1    628  36-45   male   0.0  POINT (31.302296500000004 29.836736299999988)   
2    628  36-45   male   0.0  POINT (31.302296500000004 29.836736299999988)   

   age_det main_act profile_id  vehicle_owned  trips_count  activities  \
0       39     work        420              1            6           5   
1       39     work        420              1            6           5   
2       39     work        420              1            6           5   

   repetition  
0           1  
1           2  
2           3  


In [99]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Create the 'activity_id' column
scaled_df['act_id'] = scaled_df['index'].astype(str) + '.' + scaled_df['repetition'].astype(str)

# Get the current time after the code execution
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

print("End Time:", end_time)

Start Time: 2023-06-18 11:17:59
End Time: 2023-06-18 11:18:16


In [102]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Reset the index of the scaled DataFrame
scaled_df = scaled_df.reset_index(drop=True)

# Get the current time after the code execution
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

print("End Time:", end_time)

Start Time: 2023-06-18 11:20:02
End Time: 2023-06-18 11:20:11


In [103]:
scaled_df.head(10)

,index,age,gender,educ,geometry,age_det,main_act,profile_id,vehicle_owned,trips_count,activities,repetition,act_id
0,628,36-45,male,0.0,POINT (31.302296500000004 29.836736299999988),39,work,420,1,6,5,1,628.1
1,628,36-45,male,0.0,POINT (31.302296500000004 29.836736299999988),39,work,420,1,6,5,2,628.2
2,628,36-45,male,0.0,POINT (31.302296500000004 29.836736299999988),39,work,420,1,6,5,3,628.3
3,628,36-45,male,0.0,POINT (31.302296500000004 29.836736299999988),39,work,420,1,6,5,4,628.4
4,628,36-45,male,0.0,POINT (31.302296500000004 29.836736299999988),39,work,420,1,6,5,5,628.5
5,826,<18,male,1.0,POINT (31.310103099999996 29.8016195),3,primary,1606,0,3,2,1,826.1
6,826,<18,male,1.0,POINT (31.310103099999996 29.8016195),3,primary,1606,0,3,2,2,826.2
7,850,<18,female,1.0,POINT (31.3040036 29.83853570000001),11,primary,1607,0,2,1,1,850.1
8,1171,26-35,female,0.0,POINT (31.3553709 29.84492519999999),27,work,202,0,2,1,1,1171.1
9,2019,<18,male,1.0,POINT (31.349656703769913 29.840759526980175),13,secondary,1606,0,3,2,1,2019.1


Trip distribution from OD+TI
work        56%
personal    39%
shopping     5%

In [106]:
# Create a boolean mask for rows that require updating
mask = scaled_df['activities'] < 3

# Define choices and weights for rows with activities < 3
choices_low_activities = ['main_occ', 'personal', 'shopping']
weights_low_activities = [0.56, 0.39, 0.05]

# Update 'activity' column for rows with activities < 3
scaled_df.loc[mask, 'activity'] = np.random.choice(choices_low_activities, size=mask.sum(), p=weights_low_activities)

# Create a boolean mask for rows that require further update
mask = ~mask

# Define choices and weights for rows with activities >= 3
choices_high_activities = ['main_occ', 'personal', 'shopping', 'home']
weights_high_activities = [0.48, 0.32, 0.05, 0.15]

# Get the suffix values for rows with activities >= 3
suffix_values = scaled_df.loc[mask, 'activities'].astype(str).str[-1]

# Create a boolean mask for rows where suffix values match the conditions
suffix_mask = (suffix_values == '1') | (suffix_values == scaled_df.loc[mask, 'activities'].astype(str))

# Update 'activity' column for matching rows with activities >= 3
scaled_df.loc[mask & suffix_mask, 'activity'] = np.random.choice(choices_high_activities, size=suffix_mask.sum(), p=weights_high_activities)

# Update 'activity' column for non-matching rows with activities >= 3
scaled_df.loc[mask & ~suffix_mask, 'activity'] = np.random.choice(choices_low_activities, size=(mask & ~suffix_mask).sum(), p=weights_low_activities)

[2023-06-18 11:46:25] Processed 0 rows...
[2023-06-18 11:46:25] Processed 100000 rows...
[2023-06-18 11:46:25] Processed 200000 rows...
[2023-06-18 11:46:25] Processed 300000 rows...
[2023-06-18 11:46:25] Processed 400000 rows...
[2023-06-18 11:46:25] Processed 500000 rows...
[2023-06-18 11:46:25] Processed 600000 rows...
[2023-06-18 11:46:25] Processed 700000 rows...
[2023-06-18 11:46:25] Processed 800000 rows...
[2023-06-18 11:46:25] Processed 900000 rows...
[2023-06-18 11:46:25] Processed 1000000 rows...
[2023-06-18 11:46:25] Processed 1100000 rows...
[2023-06-18 11:46:25] Processed 1200000 rows...
[2023-06-18 11:46:25] Processed 1300000 rows...
[2023-06-18 11:46:25] Processed 1400000 rows...
[2023-06-18 11:46:25] Processed 1500000 rows...
[2023-06-18 11:46:25] Processed 1600000 rows...
[2023-06-18 11:46:25] Processed 1700000 rows...
[2023-06-18 11:46:25] Processed 1800000 rows...
[2023-06-18 11:46:25] Processed 1900000 rows...
[2023-06-18 11:46:25] Processed 2000000 rows...
[2023-0

In [105]:
# # Get the current time
# start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
# print("Start Time:", start_time)
# print("Number of rows:", len(scaled_df))

# for index, row in scaled_df.iterrows():
#     if row['activities'] < 3:
#         choices = ['main_occ', 'personal', 'shopping']
#         weights = [0.56, 0.39, 0.05]
#         scaled_df.at[index, 'activity'] = np.random.choice(choices, p=weights)
#     else:
#         ends_with = [1, row['activities']]
#         if str(row['act_id']).endswith(tuple(map(str, ends_with))):
#             choices = ['main_occ', 'personal', 'shopping']
#             weights = [0.56, 0.39, 0.05]
#             scaled_df.at[index, 'activity'] = np.random.choice(choices, p=weights)
#         else:
#             choices = ['main_occ', 'personal', 'shopping', 'home']
#             weights = [0.48, 0.32, 0.05, 0.15]
#             scaled_df.at[index, 'activity'] = np.random.choice(choices, p=weights)

# # Get the current time after the code execution
# end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# print("End Time:", end_time)

In [114]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Create a boolean mask for rows where 'activity' is 'main_occ'
mask = scaled_df['activity'] == 'main_occ'

# Update the 'activity' column using vectorized operations
scaled_df.loc[mask, 'activity'] = scaled_df.loc[mask, 'main_act']

# Get the current time after the code execution
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

print("End Time:", end_time)

Start Time: 2023-06-18 11:50:20
End Time: 2023-06-18 11:50:24


In [111]:
# # Get the current time
# start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
# print("Start Time:", start_time)

# for index, row in scaled_df.iterrows():
#     if row['activity'] == 'main_occ':
#         scaled_df.at[index, 'activity'] = scaled_df.at[index, 'main_act']
        
# # Get the current time after the code execution
# end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

# print("End Time:", end_time)

### Attach trip distances

In [116]:
distmean = trips['distance'].mean()

In [117]:
# Only once 
for index, row in trips.iterrows():
    if row['trip_purpose'] == 'edu' and row['age'] != '<18':
        trips.at[index, 'trip_purpose'] = 'uni'
    elif row['trip_purpose'] == 'edu' and row['distance'] > 6:
        trips.at[index, 'trip_purpose'] = 'secondary'
    elif row['trip_purpose'] == 'edu' and row['distance'] <= 6:
        trips.at[index, 'trip_purpose'] = 'primary'

In [118]:
trips.rename(columns={'trip_purpose': 'activity'}, inplace=True)

In [119]:
trips = trips[['age','gender','activity','distance']]

In [120]:
trips

,age,gender,activity,distance
0,26-35,female,work,7.727921
1,36-45,male,work,6.529161
2,18-25,female,personal,9.302396
3,46-55,female,personal,2.162127
4,18-25,female,home,9.691204
...,...,...,...,...
11880,36-45,female,shopping,0.714051
11881,56-65,male,home,3.909081
11882,36-45,male,work,10.849445
11883,18-25,male,home,82.570406


In [126]:
# Define the chunk size
chunksize = 50000

# Create an empty list to store the merged chunks
merged_chunks = []

# Get the total number of chunks
total_chunks = -(-len(scaled_df) // chunksize)  # Round up division

# Iterate over the data in chunks
for i in range(total_chunks):
    start = i * chunksize
    end = (i + 1) * chunksize
    # Get the chunk of data
    chunk_df = scaled_df[start:end]
    # Perform the merge operation on each chunk
    merged_chunk = pd.merge(chunk_df, trips, on=['age', 'gender','activity'], how='left')    
    # Delete all but one
    merged_chunk = merged_chunk.groupby('act_id').sample(n=1).reset_index(drop=True)
    # Append the merged chunk to the list
    merged_chunks.append(merged_chunk)
    print(f'Chunk {i+1} of {total_chunks}')
    
# Concatenate the merged chunks into a single DataFrame
df_trips = pd.concat(merged_chunks)

Chunk 1 of 459
Chunk 2 of 459
Chunk 3 of 459
Chunk 4 of 459
Chunk 5 of 459
Chunk 6 of 459
Chunk 7 of 459
Chunk 8 of 459
Chunk 9 of 459
Chunk 10 of 459
Chunk 11 of 459
Chunk 12 of 459
Chunk 13 of 459
Chunk 14 of 459
Chunk 15 of 459
Chunk 16 of 459
Chunk 17 of 459
Chunk 18 of 459
Chunk 19 of 459
Chunk 20 of 459
Chunk 21 of 459
Chunk 22 of 459
Chunk 23 of 459
Chunk 24 of 459
Chunk 25 of 459
Chunk 26 of 459
Chunk 27 of 459
Chunk 28 of 459
Chunk 29 of 459
Chunk 30 of 459
Chunk 31 of 459
Chunk 32 of 459
Chunk 33 of 459
Chunk 34 of 459
Chunk 35 of 459
Chunk 36 of 459
Chunk 37 of 459
Chunk 38 of 459
Chunk 39 of 459
Chunk 40 of 459
Chunk 41 of 459
Chunk 42 of 459
Chunk 43 of 459
Chunk 44 of 459
Chunk 45 of 459
Chunk 46 of 459
Chunk 47 of 459
Chunk 48 of 459
Chunk 49 of 459
Chunk 50 of 459
Chunk 51 of 459
Chunk 52 of 459
Chunk 53 of 459
Chunk 54 of 459
Chunk 55 of 459
Chunk 56 of 459
Chunk 57 of 459
Chunk 58 of 459
Chunk 59 of 459
Chunk 60 of 459
Chunk 61 of 459
Chunk 62 of 459
Chunk 63 of 459
C

In [127]:
df_trips['distance'].fillna(distmean, inplace=True)

In [130]:
# safety stop 
df_trips.to_csv('data/interim/activitychains/population_without_times.csv')

### Attach times

In [9]:
df_trips.head(3)

,Unnamed: 0,index,age,gender,educ,geometry,age_det,main_act,profile_id,vehicle_owned,trips_count,activities,repetition,act_id,activity,distance
0,0,1000133,<18,female,1.0,POINT (31.2400075 29.7918666),12,secondary,1607,0,2,1,1,1000133.1,shopping,2.181599
1,1,10001363,36-45,male,0.0,POINT (31.27183177046704 30.11594197673126),41,work,101,0,3,2,1,10001363.1,personal,7.807914
2,2,10001363,36-45,male,0.0,POINT (31.27183177046704 30.11594197673126),41,work,101,0,3,2,2,10001363.2,work,8.054701


In [215]:
def select_trip(row):
    if row['trips_count'] == 1:
        selection = one_trip_chains.sample(n=1)
        row['start'] = selection['start_1'].values[0]
        row['end'] = selection['end_1'].values[0]   
    elif row['trips_count'] == 2:
        selection = two_trip_chains.sample(n=1)
        row['start'] = selection['start_1'].values[0]
        row['end'] = selection['end_1'].values[0]
    elif row['trips_count'] == 3:
        selection = three_trip_chains.sample(n=1)
        if row['repetition'] == 1:
            row['start'] = selection['start_0'].values[0]
            row['end'] = selection['end_0'].values[0]
        elif row['repetition'] == 2:
            row['start'] = selection['start_1'].values[0]
            row['end'] = selection['end_1'].values[0]
    elif row['trips_count'] == 4:
        selection = four_trip_chains.sample(n=1)
        if row['repetition'] == 1:
            row['start'] = selection['start_0'].values[0]
            row['end'] = selection['end_0'].values[0]
        elif row['repetition'] == 2:
            row['start'] = selection['start_1'].values[0]
            row['end'] = selection['end_1'].values[0]
        elif row['repetition'] == 3:
            row['start'] = selection['start_2'].values[0]
            row['end'] = selection['end_2'].values[0]
    elif row['trips_count'] == 5:
        selection = five_trip_chains.sample(n=1)
        if row['repetition'] == 1:
            row['start'] = selection['start_0'].values[0]
            row['end'] = selection['end_0'].values[0]
        elif row['repetition'] == 2:
            row['start'] = selection['start_1'].values[0]
            row['end'] = selection['end_1'].values[0]
        elif row['repetition'] == 3:
            row['start'] = selection['start_2'].values[0]
            row['end'] = selection['end_2'].values[0]
        elif row['repetition'] == 4:
            row['start'] = selection['start_3'].values[0]
            row['end'] = selection['end_3'].values[0]
    elif row['trips_count'] == 6:
        selection = six_trip_chains.sample(n=1)
        if row['repetition'] == 1:
            row['start'] = selection['start_0'].values[0]
            row['end'] = selection['end_0'].values[0]
        elif row['repetition'] == 2:
            row['start'] = selection['start_1'].values[0]
            row['end'] = selection['end_1'].values[0]
        elif row['repetition'] == 3:
            row['start'] = selection['start_2'].values[0]
            row['end'] = selection['end_2'].values[0]
        elif row['repetition'] == 4:
            row['start'] = selection['start_3'].values[0]
            row['end'] = selection['end_3'].values[0]
        elif row['repetition'] == 5:
            row['start'] = selection['start_4'].values[0]
            row['end'] = selection['end_4'].values[0]       
    return row

start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)
# Apply formula
activitychains = df_trips.apply(select_trip, axis=1)

end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-06-18 21:59:24
End Time: 2023-06-18 21:59:24


In [216]:
print("End Time:", end_time)

End Time: 2023-06-19 05:51:40


## Export

In [217]:
activitychains.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22946724 entries, 0 to 46723
Data columns (total 17 columns):
 #   Column         Dtype  
---  ------         -----  
 0   index          int64  
 1   age            object 
 2   gender         object 
 3   educ           float64
 4   geometry       object 
 5   age_det        int64  
 6   main_act       object 
 7   profile_id     object 
 8   vehicle_owned  int64  
 9   trips_count    int64  
 10  activities     int64  
 11  repetition     int64  
 12  act_id         object 
 13  activity       object 
 14  distance       float64
 15  start          float64
 16  end            float64
dtypes: float64(4), int64(6), object(7)
memory usage: 3.1+ GB


In [218]:
activitychains.to_csv('data/interim/activitychains/population.csv')

## Working

In [140]:
df = pd.read_csv('data/interim/activitychains/population.csv')

In [141]:
df = df.drop(['Unnamed: 0', 'profile_id', 'age_det', 'main_act', 'trips_count','educ'], axis=1)
df.rename(columns={'geometry': 'home_loc'}, inplace=True)
df.rename(columns={'vehicle_owned': 'car'}, inplace=True)
df.rename(columns={'repetition': 'act_no'}, inplace=True)
df.rename(columns={'index': 'person_id'}, inplace=True)

In [142]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Create a mask to identify rows with repetition value equal to 1
mask = df['act_no'] == 1

# Get the index positions of the rows that satisfy the condition
indices = df.index[mask]

# Duplicate rows that satisfy the condition and append them to the DataFrame
duplicated_rows = df.loc[mask].copy()

# Update the values of start and end columns in the duplicated rows
duplicated_rows['start'] = np.nan
duplicated_rows['end'] = 'tbda'

# Convert the act_id column to string data type
duplicated_rows['act_id'] = duplicated_rows['act_id'].astype(str)

# Update the last letter of the act_id column in the duplicated rows
duplicated_rows['act_id'] = duplicated_rows['act_id'].str[:-1] + '0'

# Update the activity column in the duplicated rows
duplicated_rows['activity'] = 'home'

# Update the distance column in the duplicated rows
duplicated_rows['distance'] = 0

# Update the activity number column in the duplicated rows
duplicated_rows['act_no'] = 0


df = df.append(duplicated_rows, ignore_index=True)
df['act_id'] = df['act_id'].astype(float)
df = df.sort_values(by='act_id', ascending=True)

# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-07-07 17:54:58


/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_44237/1450610785.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(duplicated_rows, ignore_index=True)


End Time: 2023-07-07 17:55:39


In [143]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# Create a mask to identify rows with repetition value equal to 1
mask = df['activities'] == df['act_no']

# Get the index positions of the rows that satisfy the condition
indices = df.index[mask]

# Duplicate rows that satisfy the condition and append them to the DataFrame
duplicated_rows = df.loc[mask].copy()

# Update the values of start and end columns in the duplicated rows
duplicated_rows['start'] = 'tbdb'
duplicated_rows['end'] = np.nan

# Convert the act_id column to string data type
duplicated_rows['act_id'] = duplicated_rows['act_id'].astype(str)

# Update the activity column in the duplicated rows
duplicated_rows['activity'] = 'home'

# Update the distance column in the duplicated rows
duplicated_rows['distance'] = 0

# Update the act_no column in the duplicated rows
duplicated_rows['act_no'] = duplicated_rows['activities'] + 1

# Update the last letter of the act_id column in the duplicated rows
duplicated_rows['act_id'] = duplicated_rows['act_id'].str[:-1] + (duplicated_rows['activities'] + 1).astype(str)

df = df.append(duplicated_rows, ignore_index=True)
df['activities'] = df['activities'] + 2
df['act_id'] = df['act_id'].astype(float)
df = df.sort_values(by='act_id', ascending=True)


# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-07-07 17:55:39


/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_44237/3811636283.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(duplicated_rows, ignore_index=True)


End Time: 2023-07-07 17:56:32


In [144]:
df.reset_index(drop=True, inplace=True)

In [24]:
# assuming 25km/h average speed

In [145]:
# calculate average distance
average_dist = df[df['activity'] != 'home']['distance'].mean()

In [146]:
# replace distances of under 1km with average distance
df.loc[(df['activity'] != 'home') & (df['distance'] < 1), 'distance'] = average_dist

# replace distances of over 100km with average distance
df.loc[df['distance'] > 100, 'distance'] = average_dist

In [147]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

home_dist_dict = []

# Create a dictionary mapping 'person_id' to the corresponding 'distance' value
home_dist_dict = df.loc[df['act_no'] == 1, ['person_id', 'distance']].set_index('person_id')['distance'].to_dict()

# Create the 'home_dist' column by mapping the values from the dictionary
df['home_dist'] = df['person_id'].map(home_dist_dict)

# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-07-07 17:56:58
End Time: 2023-07-07 17:57:17


In [148]:
safety_copy = df.copy()

In [174]:
df = safety_copy.copy()

In [171]:
# df = df.head(10000)

In [175]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

df_grouped = []

# Create the grouped DataFrame with maximum 'home_dist' and 'start' for each 'person_id' where 'act_no' is 1
df_grouped = df[df['act_no'] == 1].groupby('person_id').agg({'home_dist': 'max', 'start': 'max'}).rename(columns={'home_dist': 'home_dist_max', 'start': 'start_time'})

# # Create a new DataFrame 'df_grouped' with maximum 'home_dist' and 'start_time' for each 'person_id'
# df_grouped = df.loc[df['act_no'] == 1].groupby('person_id').max()['start']
# df_grouped = df_grouped.to_frame()
# df_grouped = df_grouped.rename(columns={'start': 'start_time'})

# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-07-07 18:04:09
End Time: 2023-07-07 18:23:03


In [176]:
# Merge the 'df_test' DataFrame with 'df_grouped' on 'person_id' to update 'home_dist' and 'start_time'
df = pd.merge(df, df_grouped, on='person_id', how='left')

In [177]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# assign first distance as home distance for intermediate or final home activity
df.loc[(df['act_no'] != 0) & (df['act_no'] != df['activities']) & (df['activity'] == 'home'), 'distance'] = df['home_dist']

# calculate end time of first home activity 
df.loc[df['act_no'] == 0, 'end'] = round(df['start_time'] - df['home_dist'] / 25 * 60)

# clean df
df.drop(columns=['home_dist','start_time'], inplace=True)

# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

Start Time: 2023-07-07 18:23:49
End Time: 2023-07-07 18:24:08


In [178]:
# Get the current time
start_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("Start Time:", start_time)

# set last activity as last_end_time
df.loc[df['activity'] != 'home', 'last_act_end'] = df['end']

grouped_df = df.groupby('person_id').max()['last_act_end']

# Get the current time
end_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
print("End Time:", end_time)

/var/folders/93/smcvnv_s04v84f_dnqvd4xkw0000gn/T/ipykernel_44237/2549207887.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  grouped_df = df.groupby('person_id').max()['last_act_end']


In [179]:
df = pd.merge(df, grouped_df, on='person_id', how='left')

In [180]:
# Calculate start time of last home activity
df.loc[
    df['act_no'] == (df.loc[:, 'activities'] - 1),
    'start'
] = round(df.loc[:, 'last_act_end_y'] + df.loc[:, 'distance'] / 25 * 60)

In [181]:
# clean df
df.drop(columns=['last_act_end_x', 'last_act_end_y'], inplace=True)

In [182]:
df = df[~(df["start"].isna() & df["act_no"] != 0)]

### Checking outputs

In [193]:
print(f'Number of trips starting before the day: {len(df[df["start"] < 0])}')
print(f'Number of trips ending before the day starts: {len(df[df["end"] < 30])}')
print(f'Number of trips ending after midnight: {len(df[df["end"] > 1440])}')
print('––––––––––––')
print(f'Number of NAs at wrong location: {len(df[(df["start"].isna() & df["act_no"] != 0)])}')

Number of trips starting before the day: 0
Number of trips ending before the day starts: 0
Number of trips ending after midnight: 0
––––––––––––
Number of NAs at wrong location: 0


In [192]:
# Deleting 11 values that end before 00:30
df = df[~(df["end"] < 30)]

In [194]:
df.to_csv('data/interim/activitychains/population+home-act.csv')

## Working 27 june

In [4]:
df = pd.read_csv('data/interim/activitychains/population+home-act.csv')

In [6]:
df = df.drop(['Unnamed: 0'], axis=1)

In [13]:
print(f"Number of individuals: {df['person_id'].nunique()}")
print(f'Number of trips: {len(df)}')

Number of individuals: 15288918
Number of trips: 53524560


In [7]:
df.head(10)

,person_id,age,gender,home_loc,car,activities,act_no,act_id,activity,distance,start,end
0,0,>75,female,POINT (31.331430700000002 29.845431899999987),1,3,0,0.0,home,0.000000,-inf,575.0
1,0,>75,female,POINT (31.331430700000002 29.845431899999987),1,3,1,0.1,personal,5.902421,589.0,1132.0
2,0,>75,female,POINT (31.331430700000002 29.845431899999987),1,3,2,0.2,home,5.902421,1146.0,inf
3,1,26-35,female,POINT (31.313874 29.8144589),1,3,0,1.0,home,0.000000,-inf,535.0
4,1,26-35,female,POINT (31.313874 29.8144589),1,3,1,1.1,work,14.937200,571.0,731.0
5,1,26-35,female,POINT (31.313874 29.8144589),1,3,2,1.2,home,14.937200,767.0,inf
6,2,26-35,female,POINT (31.3075189 29.860689800000003),1,3,0,2.0,home,0.000000,-inf,876.0
7,2,26-35,female,POINT (31.3075189 29.860689800000003),1,3,1,2.1,personal,8.715996,897.0,1179.0
8,2,26-35,female,POINT (31.3075189 29.860689800000003),1,3,2,2.2,home,8.715996,1200.0,inf
9,3,26-35,male,POINT (31.345518647215872 29.847849488673337),0,3,0,3.0,home,0.000000,-inf,498.0


In [14]:
df['activity'].unique()

array(['home', 'personal', 'work', 'primary', 'shopping', 'uni',
       'secondary'], dtype=object)